In [ ]:
# ===============================
# Stage 1
# ===============================
!apt-get -qq update && apt-get -qq install -y poppler-utils tesseract-ocr >/dev/null
!pip -q install pypdf pdf2image pytesseract nltk pandas pyarrow tqdm

# ---------- Safe (re)mount ----------
from google.colab import drive
import os, time, subprocess, shutil

MOUNTPOINT = "/content/drive"

def _is_mounted(path):
    try:
        return os.path.exists(os.path.join(path, "MyDrive"))
    except Exception:
        return False

def ensure_drive_mounted():
    if _is_mounted(MOUNTPOINT):
        return
    try:
        subprocess.run(["fusermount", "-u", MOUNTPOINT], check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception:
        pass
    time.sleep(0.5)
    shutil.rmtree(MOUNTPOINT, ignore_errors=True)
    drive.mount(MOUNTPOINT, force_remount=False)

ensure_drive_mounted()
print("✅ Drive ready.")

# ---------- Imports ----------
import re, hashlib
from collections import defaultdict
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from pypdf import PdfReader
from pdf2image import convert_from_path
import pytesseract
import nltk; nltk.download('punkt', quiet=True)
import nltk; nltk.download('punkt_tab', quiet=True)
from nltk.tokenize import sent_tokenize

# ---------- CONFIG ----------
# If this path is wrong, the code will auto-find a folder named "FORTUNE 500" under your Drive.
IN_DIR           = "<<Google Drive Folder>>/FORTUNE 500"
SKIP_EXISTING    = True
NATIVE_MIN_CHARS = 120
OCR_DPI          = 300
SENT_MINLEN      = 25

# ---------- Regex & helpers ----------
YEAR_RE     = re.compile(r'_(19|20)\d{2}(?=\.pdf$)', re.I)
PAGINATION  = re.compile(r"\b(page|pp\.|p\.|appendix|annex|exhibit|figure|table|chapter|section|see page)\b", re.I)
TOC_LINE    = re.compile(r"\.{3,}\s*\d{1,4}$")
BARE_NUM    = re.compile(r"^\s*\d{1,3}\s*$")
DATE_ONLY   = re.compile(r"^\s*(?:FY\s*\d{4}|(19|20)\d{2}|Q[1-4])\s*$", re.I)
SHORT_HEAD  = re.compile(r"^(?:contents|table of contents|index)$", re.I)

def extract_year(p: Path):
    m = YEAR_RE.search(p.name); return int(m.group(0)[1:]) if m else None

def extract_type(p: Path):
    n = p.name.upper()
    if "_AR_" in n: return "AR"
    if "_10K_" in n: return "10K"
    return "ESG"

def sha256_16(p: Path):
    import hashlib
    h=hashlib.sha256()
    with open(p,'rb') as f:
        for b in iter(lambda:f.read(1<<20), b''): h.update(b)
    return h.hexdigest()[:16]

def _keep_line(line: str) -> bool:
    s = line.strip()
    if not s: return False
    if BARE_NUM.match(s): return False
    if TOC_LINE.search(s): return False
    if SHORT_HEAD.match(s): return False
    if PAGINATION.search(s) and len(s.split()) <= 6: return False
    if DATE_ONLY.match(s): return False
    return True

def clean_lines(text: str) -> str:
    lines = text.splitlines()
    lines = [ln for ln in lines if _keep_line(ln)]
    return re.sub(r"\s+", " ", "\n".join(lines)).strip()

def to_sentences(txt: str):
    txt = clean_lines(txt)
    return [s.strip() for s in sent_tokenize(txt) if len(s.strip()) >= SENT_MINLEN]

def native_text(pdf: Path) -> str:
    try:
        pages = PdfReader(str(pdf)).pages
        parts = [(pg.extract_text() or "") for pg in pages]
        raw = "\n".join([t for t in parts if t]).strip()
        return clean_lines(raw)
    except Exception:
        return ""

def ocr_fulldoc(pdf: Path) -> str:
    try:
        imgs = convert_from_path(str(pdf), dpi=OCR_DPI)
        txts = [pytesseract.image_to_string(im, lang="eng", config="--oem 1 --psm 6").strip() for im in imgs]
        raw = "\n".join([t for t in txts if t]).strip()
        return clean_lines(raw)
    except Exception:
        return ""

# ---------- Robust resume helpers ----------
def read_manifest_safe(p: Path) -> pd.DataFrame:
    if not p.exists():
        return pd.DataFrame()
    try:
        return pd.read_parquet(p)
    except Exception:
        return pd.DataFrame()

def can_skip_pdf(filename: str, text_path: Path, sent_path: Path, existing_manifest: pd.DataFrame) -> bool:
    if text_path.exists() and sent_path.exists():
        return True
    if not existing_manifest.empty and filename in existing_manifest["filename"].values:
        row = existing_manifest.loc[existing_manifest["filename"] == filename].iloc[-1]
        txt_ok  = Path(row["text_path"]).exists()
        sent_ok = Path(row["sentences_path"]).exists()
        return txt_ok and sent_ok
    return False

def is_error_empty(filename: str, existing_manifest: pd.DataFrame) -> bool:
    if existing_manifest.empty or filename not in existing_manifest["filename"].values:
        return False
    status = existing_manifest.loc[existing_manifest["filename"] == filename].iloc[-1]["status"]
    return status == "error_empty"

# ---------- Locate the root folder robustly ----------
MYDRIVE = Path("/content/drive/MyDrive")
in_root = Path(IN_DIR)

if not in_root.exists():
    print(f"⚠️ Provided IN_DIR not found:\n  {in_root}")
    # Try a quick heuristic (Award → Awards)
    alt = Path(str(in_root).replace("Australia Award Scholarship", "Australia Awards Scholarship"))
    if alt.exists():
        in_root = alt
        print(f"➡️ Using alternate path:\n  {in_root}")
    else:
        # Search for a folder literally named "FORTUNE 500" under MyDrive
        print("🔎 Searching for a folder named 'FORTUNE 500' under MyDrive ...")
        candidates = [p for p in MYDRIVE.rglob("*") if p.is_dir() and p.name == "FORTUNE 500"]
        if not candidates:
            # Also try case-insensitive match, just in case
            candidates = [p for p in MYDRIVE.rglob("*") if p.is_dir() and p.name.lower() == "fortune 500"]
        if candidates:
            in_root = candidates[0]
            print(f"✅ Found:\n  {in_root}")
        else:
            raise FileNotFoundError("Could not locate 'FORTUNE 500' folder. Please check the path or rename IN_DIR.")

# ---------- Discover PDFs case-insensitively & group by company folder ----------
all_paths = list(in_root.rglob("*"))
all_pdfs = [p for p in all_paths if p.is_file() and p.suffix.lower() == ".pdf"]

# Sanity preview
print(f"\n📂 Using root: {in_root}")
print(f"🔢 PDFs discovered: {len(all_pdfs)}")
print("👀 First 10 PDFs:")
for p in all_pdfs[:10]:
    print(" -", p)

groups = defaultdict(list)
for pdf in all_pdfs:
    groups[pdf.parent].append(pdf)

print(f"\n🏢 Company folders with PDFs: {len(groups)}")
print("👀 First 10 folders:")
for i, folder in enumerate(list(groups.keys())[:10]):
    print(" -", folder)
print()

# ---------- Process per folder ----------
total_processed = 0
total_skipped   = 0
per_folder_summaries = []

for folder, pdfs in groups.items():
    out_root = folder / "esg_stage1"
    texts_dir = out_root / "texts"
    sents_dir = out_root / "sentences"
    texts_dir.mkdir(parents=True, exist_ok=True)
    sents_dir.mkdir(parents=True, exist_ok=True)

    manifest_path = out_root / "manifest.parquet"
    existing = read_manifest_safe(manifest_path)

    rows = []
    skipped_here = 0
    processed_here = 0

    for pdf in tqdm(pdfs, desc=f"Processing PDFs in {folder.name}", leave=False):
        filename = pdf.name
        hid = sha256_16(pdf)
        text_path = texts_dir / f"{hid}.txt"
        sent_path = sents_dir / f"{hid}.parquet"

        if SKIP_EXISTING and can_skip_pdf(filename, text_path, sent_path, existing) and not is_error_empty(filename, existing):
            skipped_here += 1
            continue

        year = extract_year(pdf); rtype = extract_type(pdf)

        # 1) Native
        txt = native_text(pdf)
        if txt and len(txt) >= NATIVE_MIN_CHARS:
            status = "ok_native"
        else:
            # 2) Full-doc OCR
            txt = ocr_fulldoc(pdf)
            status = "ok_ocr" if txt and len(txt) >= NATIVE_MIN_CHARS else "error_empty"

        # Persist (even empty to keep manifest consistent)
        text_path.write_text(txt or "", encoding="utf-8")
        pd.DataFrame({"text": to_sentences(txt) if txt else []}).to_parquet(sent_path, index=False)

        rows.append({
            "sha256_16": hid,
            "filename": filename,
            "pdf_path": str(pdf),
            "text_path": str(text_path),
            "sentences_path": str(sent_path),
            "status": status,
            "doc_year": year,
            "report_type": rtype
        })
        processed_here += 1

    # Merge+save manifest
    if rows:
        manifest = (pd.concat([existing, pd.DataFrame(rows)], ignore_index=True)
                    if not existing.empty else pd.DataFrame(rows))
        manifest = manifest.drop_duplicates(subset=["filename"], keep="last")
        manifest.to_parquet(manifest_path, index=False)
    else:
        manifest = existing if not existing.empty else pd.DataFrame(columns=[
            "sha256_16","filename","pdf_path","text_path","sentences_path","status","doc_year","report_type"
        ])

    per_folder_summaries.append({
        "folder": str(folder),
        "pdfs_found": len(pdfs),
        "processed_now": processed_here,
        "skipped_existing": skipped_here,
        "manifest_path": str(manifest_path),
        "manifest_rows": len(manifest)
    })
    total_processed += processed_here
    total_skipped += skipped_here

# ---------- Summary ----------
print("\n✅ Done.")
print(f"Folders: {len(groups)}  |  PDFs total: {len(all_pdfs)}")
print(f"Processed now: {total_processed}  |  Skipped (existing): {total_skipped}\n")
for s in per_folder_summaries[:15]:
    print(f"- {Path(s['folder']).name}: found={s['pdfs_found']}, processed={s['processed_now']}, "
          f"skipped={s['skipped_existing']}, manifest_rows={s['manifest_rows']}")
    print(f"  → {s['manifest_path']}")
